In [ ]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model
import RPi.GPIO as GPIO

# GPIO 핀 번호 설정
red_pin = 17  # 빨간색 LED에 연결된 GPIO 핀 번호
green_pin = 18  # 초록색 LED에 연결된 GPIO 핀 번호
blue_pin = 27  # 파란색 LED에 연결된 GPIO 핀 번호
trig_pin = 23  # 초음파 센서 Trig 핀
echo_pin = 24  # 초음파 센서 Echo 핀
in1_pin = 19
in2_pin = 20
in3_pin = 21
in4_pin = 22
ena_pin = 5
enb_pin = 6

# GPIO 설정을 BCM 모드로 합니다.
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# GPIO 핀을 입/출력으로 설정합니다.
GPIO.setup(red_pin, GPIO.OUT)
GPIO.setup(green_pin, GPIO.OUT)
GPIO.setup(blue_pin, GPIO.OUT)
GPIO.setup(trig_pin, GPIO.OUT)
GPIO.setup(echo_pin, GPIO.IN)
GPIO.setup(in1_pin, GPIO.OUT)
GPIO.setup(in2_pin, GPIO.OUT)
GPIO.setup(in3_pin, GPIO.OUT)
GPIO.setup(in4_pin, GPIO.OUT)
GPIO.setup(ena_pin, GPIO.OUT)
GPIO.setup(enb_pin, GPIO.OUT)

# LED 색상을 제어하는 함수
def set_led_color(red, green, blue):
    GPIO.output(red_pin, red)
    GPIO.output(green_pin, green)
    GPIO.output(blue_pin, blue)

# 예측 결과에 따라 LED 색상을 설정하는 함수
def set_led_color_by_result(result):
    if result == 'fire':
        set_led_color(1, 0, 0)  # 빨간색 LED 켜기
    elif result == 'human':
        set_led_color(0, 1, 0)  # 초록색 LED 켜기
    elif result == 'water':
        set_led_color(0, 0, 1)  # 파란색 LED 켜기
    else:
        set_led_color(1, 1, 1)  # 모든 LED 켜기

# 학습된 모델을 로드합니다.
model_save_path = '/home/jemin/Downloads/model_trained_data.h5'
model = load_model(model_save_path)

# 모터 제어 함수
def control_motor(in1_state, in2_state, in3_state, in4_state, ena_duty_cycle, enb_duty_cycle):
    GPIO.output(in1_pin, in1_state)
    GPIO.output(in2_pin, in2_state)
    GPIO.output(in3_pin, in3_state)
    GPIO.output(in4_pin, in4_state)
    GPIO.output(ena_pin, GPIO.HIGH)
    GPIO.output(enb_pin, GPIO.HIGH)

    pwm_ena.ChangeDutyCycle(ena_duty_cycle)
    pwm_enb.ChangeDutyCycle(enb_duty_cycle)

# PWM(펄스 폭 변조) 설정
pwm_frequency = 1000
pwm_ena = GPIO.PWM(ena_pin, pwm_frequency)
pwm_enb = GPIO.PWM(enb_pin, pwm_frequency)

pwm_ena.start(0)
pwm_enb.start(0)

# 초음파 센서로 거리를 측정하는 함수
def measure_distance():
    GPIO.output(trig_pin, GPIO.LOW)
    time.sleep(0.5)
    GPIO.output(trig_pin, GPIO.HIGH)
    time.sleep(0.00001)
    GPIO.output(trig_pin, GPIO.LOW)

    while GPIO.input(echo_pin) == 0:
        pulse_start = time.time()
    while GPIO.input(echo_pin) == 1:
        pulse_end = time.time()

    pulse_duration = pulse_end - pulse_start
    distance = pulse_duration * 17150
    distance = round(distance, 2)

    return distance

# 웹캠을 시작합니다.
cap = cv2.VideoCapture(0)
# 웹캠에서 영상을 읽어 모델로 예측을 수행하고 그 결과에 따라 LED와 모터를 제어하는 루프
while True:
    ret, frame = cap.read()
    if ret:
        # 웹캠의 이미지를 전처리합니다.
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        gray = cv2.resize(gray, (128, 128))
        gray = gray / 255.0
        input_img = gray.reshape(1, 128, 128, 1)

        prediction = model.predict(input_img)
        probability = np.amax(prediction)
        class_index = np.argmax(prediction)

        # 모델의 예측 결과를 해석합니다.
        if class_index == 0:
            label = 'fire'
        elif class_index == 1:
            label = 'human'
        elif class_index == 2:
            label = 'water'
        else:
            continue

        # 화면에 예측 결과를 표시합니다.
        if probability >= 0.95:
            text = f'{label} ({probability * 100:.2f}%)'
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        else:
            text = 'Low probability'
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        set_led_color_by_result(label if probability >= 0.95 else '')

        cv2.imshow('Webcam', frame)

        # 초음파 센서로 거리를 측정하고, 이 거리와 예측 결과에 따라 모터를 제어합니다.
        distance = measure_distance()
        if probability >= 0.95 and distance < 15:
            control_motor(GPIO.LOW, GPIO.HIGH, GPIO.HIGH, GPIO.LOW, 0, 0)  
        else:
            control_motor(GPIO.HIGH, GPIO.LOW, GPIO.HIGH, GPIO.LOW, 0, 0)
    else:
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
GPIO.cleanup()